In [1]:
## Importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from constants import LINKEDIN_USER_EMAIL, LINKEDIN_USER_PASS
import numpy as np
import pandas as pd

In [2]:
# Creating an instance of Chrome WebDriver
## WebDriver Version is 120
driver = webdriver.Chrome("chromedriver.exe")

C:\Users\user\AppData\Local\Temp\ipykernel_27708\2909792738.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [3]:
## A function to scroll down the web-page
def scroll_down():
    start = time.time()

    # will be used in the while loop
    initialScroll = 0
    finalScroll = 1000

    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll 
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 1000
        # we will stop the script for 3 seconds so that 
        # the data can load
        time.sleep(3)
        # You can change it as per your needs and internet speed
        end = time.time()
        # We will scroll for 20 seconds.
        # You can change it as per your needs and internet speed
        if round(end - start) > 20:
            break

In [4]:
## Importing Necessary Libraries and creating an instance of Linkedin Class to get the links from a search query
from linkedin_api import Linkedin
from constants import LINKEDIN_USER_EMAIL, LINKEDIN_USER_PASS

linkedin_api = Linkedin(LINKEDIN_USER_EMAIL, LINKEDIN_USER_PASS)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
## Assuming first and last names of any uset is given
First_name = 'YOUR_FIRST_NAME'
Last_name = 'YOUR_LAST_NAME'

In [6]:
## Creating the name (in lower case)
name = First_name + ' ' + Last_name
name = name.lower()

In [7]:
## Searching for the profile with the keyword as the name
profiles = linkedin_api.search(params={'keywords': name}, limit=1)
## Creating users' table
df = pd.DataFrame(profiles)
no_of_profiles = 10
## Selecting top 10 links of linkedin profiles from the list
links_of_profiles = list(df['navigationUrl'][:no_of_profiles])

In [8]:
profiles_data = []
# Logging into LinkedIn
driver.get("https://linkedin.com/uas/login")
time.sleep(5)

## Entering the Email ID
username = driver.find_element(By.ID, "username")
username.send_keys(LINKEDIN_USER_EMAIL)
## Entering the Password
pword = driver.find_element(By.ID, "password")
pword.send_keys(LINKEDIN_USER_PASS)	 # Enter Your Password
## Submitting the login request
driver.find_element(By.XPATH, "//button[@type='submit']").click()
## Going through each profile
for link in links_of_profiles :
    data = {}
    # Trying to Open Profile
    try :
        profile_url = link
        driver.get(profile_url)	 # this will open the link
        scroll_down()	 # this will scroll down the page

        src = driver.page_source
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # Extracting the HTML of the complete introduction box
        # that contains the name, company name, and the location
        ## name
        name = soup.find('h1', {'class' : 'text-heading-xlarge inline t-24 v-align-middle break-words'})
        data['name'] = name.text if name is not None else np.nan 
        ## Headline
        tag = soup.find('div', {'class': 'text-body-medium break-words'})
        data['headline'] = tag.text.replace('\n', '').strip() if tag is not None else np.nan
        ## Place
        loc = soup.find('span', {'class': 'text-body-small inline t-black--light break-words'})
        data['location'] = loc.text.replace('\n', '').strip() if loc is not None else np.nan
        ## About
        about = soup.find('div', {'class': 'display-flex ph5 pv3'})
        data['about'] = about.span.text.replace('\n', '').strip() if about is not None else np.nan
    except Exception as e:
        print("Profile not found...")
        data['name'] = np.nan
        data['headline'] = np.nan
        data['location'] = np.nan
        data['about'] = np.nan

    ## Trying to Scrape Experiences
    try :
        experience_url = profile_url + "details/experience/"
        driver.get(experience_url)  # this will open the link
        scroll_down()	 # this will scroll down the page
        src = driver.page_source
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # Extracting the HTML of the complete experiece box
        # that contains experiences
        roles = soup.findAll('div', {'class': 'display-flex align-items-center mr1 t-bold'})
        companies = soup.findAll('span', {'class': 't-14 t-normal'})
        dates = soup.findAll('span', {'class': 'pvs-entity__caption-wrapper'})
        ## Storing them in a list of dictionaries format
        final_experiences = []
        for i in range(0, len(roles)) :
            experiences = {}
            experiences['role'] = (roles[i].span.text if len(roles) > i else '')
            experiences['company'] = (companies[i].span.text.replace('\n', '').strip() if len(companies) > i else '')
            experiences['date'] = (dates[i].text if len(dates) > i else '')
            final_experiences.append(experiences)
            
        data['experiences'] = [final_experiences]
    except Exception as e:
        print("Experiences not found...")
        data['experiences'] = [np.nan]

    ## Trying to Scrape Education
    try :
        education_url = profile_url + "details/education/"
        driver.get(education_url)	 # this will open the link
        scroll_down()	 # this will scroll down the page

        src = driver.page_source
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # Extracting the HTML of the education box
        schools = soup.findAll('div', {'class': 'display-flex align-items-center mr1 hoverable-link-text t-bold'})
        degrees = soup.findAll('span', {'class': 't-14 t-normal'})
        dates = soup.findAll('span', {'class': 'pvs-entity__caption-wrapper'})

        final_education = []
        for i in range(0, len(schools)) :
            education = {}
            education['role'] = (schools[i].span.text if len(schools) > i else '')
            education['company'] = (degrees[i].span.text.replace('\n', '').strip() if len(degrees) > i else '')
            education['date'] = (dates[i].text if len(dates) > i else '')
            final_education.append(education)
            
        data['experiences'] = [final_education]
    except Exception as e:
        print("Education not found...")
        data['education'] = [np.nan]

    ## Trying to Scrape skills section
    try :
        skill_url = profile_url + 'details/skills/'

        driver.get(skill_url)	 # this will open the link
        scroll_down()	 # this will scroll down the page

        src = driver.page_source
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # Extracting the HTML of the skills box
        skill_elements = soup.findAll('div', {'class': 'display-flex flex-row justify-space-between'})
        skills = []
        for i in range(0, len(skill_elements)) :
            skills.append(skill_elements[i].span.text)

        data['skills'] = [skills]
    except Exception as e:
        print("Skills not found...")
        data['skills'] = [np.nan]
    ## Storing the data of a person in a list
    profiles_data.append(data)
    print("Finished Scraping A Profile...!!!")
    ## Done with a profile

In [10]:
## Storing the data in local
linkedin_profiles_data = pd.DataFrame(profiles_data)
filename = First_name + '_' + Last_name + '.csv'
linkedin_profiles_data.to_csv(filename, index=False)